#### Preliminaries

First, let’s connect to our instance of Memgraph with gqlalchemy and load the dataset.

In [1]:
from gqlalchemy import Memgraph

In [2]:
def load_dataset(path: str):
    with open(path, mode='r') as dataset:
        for statement in dataset:
            memgraph.execute(statement)

In [3]:
memgraph = Memgraph("127.0.0.1", 7687)    # connect to running instance
memgraph.drop_database()                  # make sure it’s empty
load_dataset('data/input.cyp')            # load dataset

#### Example

With everything set up, calling the `betweenness_centrality_online` module is a matter of a single Cypher query. 
As we are analyzing how changes affect the undersea internet cable network, we save the computed betweenness centrality scores for later. 

In [4]:
memgraph.execute(
    """
    CALL betweenness_centrality_online.set()
    YIELD node, betweenness_centrality
    SET node.bc = betweenness_centrality;
    """
)

Let’s see which landing points have the highest betweenness centrality score in the network:

In [5]:
most_central = memgraph.execute_and_fetch(
    """
    MATCH (n: Node)
    RETURN n.id AS id, n.name AS name, n.bc AS bc_score
    ORDER BY bc_score DESC, name ASC
    LIMIT 5;
    """
)
for node in most_central:
    print(node)

{'id': 15, 'name': 'Tuas, Singapore', 'bc_score': 0.29099145440251284}
{'id': 16, 'name': 'Fortaleza, Brazil', 'bc_score': 0.1380757216343068}
{'id': 467, 'name': 'Toucheng, Taiwan', 'bc_score': 0.13361801370831092}
{'id': 62, 'name': 'Manado, Indonesia', 'bc_score': 0.12915295031722296}
{'id': 123, 'name': 'Balboa, Panama', 'bc_score': 0.127837144605276}


Two of the above results, Singapore and Panama, have become international trade hubs owing to their favorable geographic position. They are highly influential nodes in other networks as well.

#### Dynamic operation

This part brings us to MAGE’s newest algorithm – [**iCentral**](https://repository.kaust.edu.sa/bitstream/handle/10754/625935/08070346.pdf) dynamic betweenness centrality by [Jamour](http://www.fuadjamour.com/) et al.
After each graph update, iCentral can be run to update previously computed values without having to process the entire graph, going hand in hand with Memgraph’s **graph streaming** capabilities.

You can set this up in Memgraph with [**triggers**](https://memgraph.com/docs/memgraph/reference-guide/triggers) – pieces of Cypher code that run after database transactions.

In [6]:
memgraph.execute(
    """
    CREATE TRIGGER update_bc 
    BEFORE COMMIT EXECUTE 
        CALL betweenness_centrality_online.update(createdVertices, createdEdges, deletedVertices, deletedEdges) 
        YIELD *;
    """
)

Let’s see now what happens when a shark (or something else) cuts off a submarine internet cable.

![](images/shark.png)

In [7]:
memgraph.execute("""MATCH (n {name: "Tuas, Singapore"})-[e]-(m {name: "Jeddah, Saudi Arabia"}) DELETE e;""")

The above transaction activates the `update_bc` trigger, and previously computed betweenness centrality scores are updated using iCentral.

With the cable out of function, internet data must be transmitted over different routes. Some nodes in the network are bound to experience increased strain and internet speed might thus deteriorate. These nodes likely saw their betweenness centrality score increase. To inspect that, we’ll retrieve the new scores with `betweenness_centrality_online.get()` and compare them with the previously saved ones.

In [8]:
highest_deltas = memgraph.execute_and_fetch(
    """
    CALL betweenness_centrality_online.get()
    YIELD node, betweenness_centrality
    RETURN 
        node.id AS id,
        node.name AS name, 
        node.bc AS old_bc,
        betweenness_centrality AS bc,
        betweenness_centrality - node.bc AS delta
    ORDER BY abs(delta) DESC, name ASC
    LIMIT 5;
    """
)
for result in highest_deltas:
    print(result)

memgraph.execute("DROP TRIGGER update_bc;")

{'id': 111, 'name': 'Jeddah, Saudi Arabia', 'old_bc': 0.061933737931979434, 'bc': 0.004773934386713372, 'delta': -0.057159803545266064}
{'id': 352, 'name': 'Songkhla, Thailand', 'old_bc': 0.05259842296405679, 'bc': 0.07514804741735284, 'delta': 0.02254962445329605}
{'id': 15, 'name': 'Tuas, Singapore', 'old_bc': 0.29099145440251284, 'bc': 0.2730690696075149, 'delta': -0.017922384794997914}
{'id': 175, 'name': 'Yanbu, Saudi Arabia', 'old_bc': 0.06483588246822349, 'bc': 0.0756199291423186, 'delta': 0.010784046674095119}
{'id': 210, 'name': 'Dakar, Senegal', 'old_bc': 0.08708567541545127, 'bc': 0.0941236276148527, 'delta': 0.007037952199401426}


As seen above, the network landing point in Songkhla, Thailand had its score increase by **42.87%** after the update. Conversely, other landing points became less connected to the rest of the network: the centrality of the Jeddah node in Saudi Arabia almost dropped to zero.

#### Performance

iCentral builds upon the Brandes algorithm and adds the following improvements in order to improve performance:
* **Process only the nodes whose betweenness centrality values change**: after an update, betweenness centrality scores stay the same outside the affected [biconnected component](https://memgraph.com/docs/mage/algorithms/traditional-graph-analytics/biconnected-components-algorithm).
* **Avoid repeating shortest-path calculations**: use prior output if it’s possible to tell it’s still valid; if new shortest paths are needed, update the prior ones instead of recomputing.
    * Breadth-first search for computing graph dependencies does not need to be done out of nodes equidistant to both endpoints of the updated edge.
    * The BFS tree used for computing new graph dependencies (the contributions of a node to other nodes’ scores) can be determined from the tree obtained while computing old graph dependencies.

In [9]:
bcc_partition = memgraph.execute_and_fetch(
    """
    CALL biconnected_components.get()
    YIELD bcc_id, node_from, node_to
    RETURN
        bcc_id,
        node_from.id AS from_id,
        node_from.name AS from_name,
        node_to.id AS to_id,
        node_to.name AS to_name
    LIMIT 15;
    """
)
for relationship in bcc_partition:
    print(relationship)

{'bcc_id': 0, 'from_id': 10, 'from_name': 'Telisai, Brunei', 'to_id': 11, 'to_name': 'Chung Hom Kok, China'}
{'bcc_id': 1, 'from_id': 695, 'from_name': 'Wenchang, China', 'to_id': 696, 'to_name': 'Zhuhai, China'}
{'bcc_id': 2, 'from_id': 11, 'from_name': 'Chung Hom Kok, China', 'to_id': 695, 'to_name': 'Wenchang, China'}
{'bcc_id': 3, 'from_id': 72, 'from_name': 'Agats, Indonesia', 'to_id': 73, 'to_name': 'Baa, Indonesia'}
{'bcc_id': 4, 'from_id': 73, 'from_name': 'Baa, Indonesia', 'to_id': 74, 'to_name': 'Kep. Aru, Indonesia'}
{'bcc_id': 5, 'from_id': 74, 'from_name': 'Kep. Aru, Indonesia', 'to_id': 75, 'to_name': 'Kokar, Indonesia'}
{'bcc_id': 6, 'from_id': 75, 'from_name': 'Kokar, Indonesia', 'to_id': 76, 'to_name': 'Kota Mappi, Indonesia'}
{'bcc_id': 7, 'from_id': 76, 'from_name': 'Kota Mappi, Indonesia', 'to_id': 77, 'to_name': 'Kupang, Indonesia'}
{'bcc_id': 8, 'from_id': 77, 'from_name': 'Kupang, Indonesia', 'to_id': 838, 'to_name': 'Alor, Indonesia'}
{'bcc_id': 9, 'from_id': 73